# LSSTCam pointing errors

Craig Lage - 16-Apr-25

In [ ]:
import os
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
from astropy.time import Time, TimeDelta
from lsst.daf.butler import Butler
import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.summit.utils.utils import dayObsIntToString
from lsst.summit.utils.efdUtils import calcNextDay
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData

In [ ]:
butler = Butler('/repo/embargo', collections=['LSSTCam/raw/all', 
                                            'LSSTCam/calib/unbounded', 'LSSTCam/runs/nightlyValidation', 
                                             'LSSTCam/runs/nightlyValidation/20250425/w_2025_17/DM-50157'])
instrument = 'LSSTCam'
client = makeEfdClient()

In [ ]:
startDay = 20250527
endDay = 20250608

els = []
azs = []
ras = []
decs = []
rots = []
deltaRas = []
deltaDecs = []
dayObss = []
filters = []
pressures = []
temps = []
hums = []
times = []
dayObs = startDay
while dayObs <= endDay:
    exposureList = []
    for record in butler.registry.queryDimensionRecords("exposure", 
                where=f"exposure.day_obs={dayObs} and instrument='LSSTCam'"):
        exposureList.append([record.id, record])
    exposureList.sort(key=lambda x: x[0])
    print(len(exposureList))
    for [id,record] in exposureList:
        if record.observation_type not in ['science', 'acq']:
            continue
        try:
            calExp = butler.get('preliminary_visit_image', detector=94, visit=record.id, instrument=instrument)
            rawExp = butler.get('raw', detector=94, exposure=record.id, instrument=instrument)
            md = rawExp.getMetadata()
            cWcs = calExp.getWcs()
            rWcs = rawExp.getWcs()
            rawSkyCenter = rWcs.getSkyOrigin()
            calExpSkyCenter = cWcs.pixelToSky(rWcs.getPixelOrigin())
            ra = calExpSkyCenter.getRa().asDegrees()
            dec = calExpSkyCenter.getDec().asDegrees()
            #print(ra, dec)
            deltaRa = rawSkyCenter.getRa().asArcseconds() - calExpSkyCenter.getRa().asArcseconds()
            deltaDec = rawSkyCenter.getDec().asArcseconds() - calExpSkyCenter.getDec().asArcseconds()
            deltaRas.append(deltaRa)
            deltaDecs.append(deltaDec)
            els.append(md['ELSTART'])
            azs.append(md['AZSTART'])
            rots.append(md['ROTPA'])
            ras.append(ra)
            decs.append(dec)
            filters.append(md['FILTBAND'])
            pressures.append(md['PRESSURE'])
            temps.append(md['AIRTEMP'])
            hums.append(md['HUMIDITY'])
            times.append((md['MJD-BEG'] + md['MJD-END']) / 2.0)
            dayObss.append(dayObs)
            print(f"{record.id}, deltaRa = {deltaRa:.2f}, deltaDec = {deltaDec:.2f}")
        except:
            print(f"{record.id} failed!")
            continue
    print(dayObs, len(ras))
    dayObs = calcNextDay(dayObs)

filename = "/home/c/cslage/u/LSSTCam/data/pointing_results_rot_dayObs_27may25-08jun25.pkl"
with open(filename, 'wb') as f:
    pkl.dump([els, azs, ras, decs, rots, deltaRas, deltaDecs, pressures, temps, hums, times, filters, dayObss], f)


In [ ]:
filename = "/home/c/cslage/u/LSSTCam/data/pointing_results_rot_dayObs_27may25-08jun25.pkl"
with open(filename, 'rb') as f:
    [els, azs, ras, decs, rots, deltaRas, deltaDecs, pressures, temps, hums, times, filters, dayObss] = pkl.load(f)
len(els)

In [ ]:
print(len(ras), len(decs), len(els), len(azs), len(rots), len(pressures), len(temps), len(hums), len(times), len(filters), len(dayObss))

In [ ]:
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5, SkyCoord
import astropy.units as u
from lsst.obs.lsst.translators.lsst import SIMONYI_LOCATION
wavelengths = {'u':3671, 'g':4827, 'r':6223, 'i':7546, 'z':8691, 'y':9712}

In [ ]:
diffs = []
for index in range(len(ras)):
    skyLocation = SkyCoord(ras[index] * u.deg, decs[index] * u.deg)
    tel_skyLocation = SkyCoord((ras[index] + deltaRas[index] / 3600.0) * u.deg, (decs[index] + deltaDecs[index] / 3600.0) * u.deg)
    separation_angle = skyLocation.separation(tel_skyLocation).arcsec
    diffs.append(separation_angle)


In [ ]:
plt.hist(diffs)
plt.title("Pointing error separation on sky (arcsec)")
plt.xlabel("Arcseconds")
plt.text(100, 1000, f"Median error = {np.median(diffs):.1f} arcseconds")
plt.savefig("/home/c/cslage/u/LSSTCam/data/Error_on_Sky_12Jun25.png")

In [ ]:
deltaAzs = []
deltaEls = []
trueAzs = []
trueEls = []
bad_indices = []
for index in range(len(ras)):
    skyLocation = SkyCoord(ras[index]*u.deg, decs[index]*u.deg)
    time = Time(times[index], format='mjd', scale='tai')
    pressure = pressures[index] * u.pascal
    temperature = temps[index] * u.Celsius
    hum = hums[index]
    wl = wavelengths[filters[index]] * u.angstrom
    altAz = AltAz(obstime=time, location=SIMONYI_LOCATION, pressure=pressure, 
                 temperature=temperature, relative_humidity=hum, obswl=wl)
    obsAltAz = skyLocation.transform_to(altAz)
    trueAz = obsAltAz.az.deg
    trueEl = obsAltAz.alt.deg
    if azs[index] > 180.0:
        azs[index] -= 360.0
    if azs[index] < -180.0:
        azs[index] += 360.0
    if trueAz > 180.0:
        trueAz -= 360.0
    if trueAz < -180.0:
        trueAz += 360.0

    deltaAz = azs[index] - obsAltAz.az.deg
    if deltaAz > 180.0:
        deltaAz -= 360.0
    if deltaAz < -180.0:
        deltaAz += 360.0
    trueAzs.append(trueAz)
    trueEls.append(trueEl)
    deltaAz *= 3600.0# * np.cos(obsAltAz.alt.rad)
    deltaEl = (els[index] - obsAltAz.alt.deg) * 3600.0
    deltaAzs.append(deltaAz)
    deltaEls.append(deltaEl)
"""    
    if abs(deltaEl) < 450.0:
        deltaAzs.append(deltaAz)
        deltaEls.append(deltaEl)
    else:
        print(index)
        bad_indices.append(index)
    

for index in bad_indices:
    del azs[index]
    del els[index]

"""
len(els)        

In [ ]:
plt.figure(figsize=(12,8))
plt.subplots_adjust(hspace=0.3, wspace=0.3)
plt.subplot(1,2,1, aspect=1)
plt.scatter(azs, (np.array(trueAzs) + np.array(deltaAzs) / 3600.0))
plt.subplot(1,2,2, aspect=1)
plt.scatter(els, (np.array(trueEls) + np.array(deltaEls) / 3600.0))


In [ ]:
print(len(ras), len(decs), len(els), len(deltaEls), len(azs), len(deltaAzs), len(rots))

In [ ]:
filename = "/home/c/cslage/u/LSSTCam/data/pointing_results_engtest_13may25.pkl"
with open(filename, 'rb') as f:
    [els2, azs2, ras, decs, deltaRas, deltaDecs, pressures, temps, hums, times, filters] = pkl.load(f)
len(els2)

In [ ]:
els += els2
azs += azs2
len(els)

In [ ]:
for index in range(len(els2)):
    skyLocation = SkyCoord(ras[index]*u.deg, decs[index]*u.deg)
    time = Time(times[index], format='mjd', scale='tai')
    pressure = pressures[index] * u.pascal
    temperature = temps[index] * u.Celsius
    hum = hums[index]
    wl = wavelengths[filters[index]] * u.angstrom
    altAz = AltAz(obstime=time, location=SIMONYI_LOCATION, pressure=pressure, 
                 temperature=temperature, relative_humidity=hum, obswl=wl)
    obsAltAz = skyLocation.transform_to(altAz)
    deltaAz = azs2[index] - obsAltAz.az.deg
    trueAzs.append(obsAltAz.az.deg)
    trueEls.append(obsAltAz.alt.deg)
    if deltaAz > 360.0:
        deltaAz -= 360.0
    if deltaAz < -180.0:
        deltaAz += 360.0
    deltaAz *= 3600.0 * np.cos(obsAltAz.alt.rad)
    deltaEl = (els2[index] - obsAltAz.alt.deg) * 3600.0
    deltaAzs.append(deltaAz)
    deltaEls.append(deltaEl)
"""    
    if abs(deltaEl) < 450.0:
        deltaAzs.append(deltaAz)
        deltaEls.append(deltaEl)
    else:
        print(index)
        bad_indices.append(index)
    

for index in bad_indices:
    del azs[index]
    del els[index]

"""
len(deltaEls)        

In [ ]:
plt.figure(figsize=(8,8))
plt.suptitle("LSSTCam Delta RA/Dec 20250527-20250608")
plt.subplots_adjust(hspace=0.3, wspace=0.3)
plt.subplot(2,2,1)
plt.scatter(decs, deltaDecs)
plt.xlabel('Dec (deg)')
plt.xlim(-90, 0)
plt.ylabel('Delta Dec arcsec')
plt.subplot(2,2,2)
plt.scatter(ras, deltaDecs)
plt.xlabel('Ra (deg)')
plt.xlim(0,360)
plt.ylabel('Delta Dec arcsec')
plt.subplot(2,2,3)
plt.scatter(decs, deltaRas)
plt.xlabel('Dec (deg)')
plt.xlim(-90, 0)
plt.ylim(-100, 500)
plt.ylabel('Delta Ra arcsec')
plt.subplot(2,2,4)
plt.scatter(ras, deltaRas)
plt.xlabel('Ra (deg)')
plt.xlim(0,360)
plt.ylim(-100, 500)
plt.ylabel('Delta Ra arcsec')
plt.savefig("/home/c/cslage/u/LSSTCam/data/Delta_RaDec_12Jun25.png")


In [ ]:
from scipy.optimize import minimize
def FOM1(params, args):
    fom = 0.0
    [azs, deltaEls] = args
    [amp, off] = params
    for i in range(len(azs)):
        if abs(deltaEls[i]) > 450.0:
            continue
        model = off + amp * np.sin((azs[i]) * np.pi / 180.0)
        err = np.square(model - deltaEls[i])
        fom += err
    return fom

args = [azs, deltaEls]
x0 = [400.0, 0.0]
result1 = minimize(FOM1, x0, args=args, method='Powell')
result1

In [ ]:
from scipy.optimize import minimize
def FOM2(params, args):
    fom = 0.0
    [azs, deltaAzs] = args
    [amp, off] = params
    for i in range(len(azs)):
        if abs(deltaAzs[i]) > 450.0:
            continue
        model = off + amp * np.cos((azs[i]) * np.pi / 180.0)
        err = np.square(model - deltaAzs[i] * np.cos((els[i]) * np.pi / 180.0))
        fom += err
    return fom

args2 = [azs, deltaAzs]
x0 = [400.0, 100.0]
result2 = minimize(FOM2, x0, args=args2, method='Powell')
result2

In [ ]:
def errs1(amp1, off1, azs, deltaEls):
    errs = []
    for i in range(len(azs)):
        model = off1 + amp1 * np.sin((azs[i]) * np.pi / 180.0)
        err = abs(model - deltaEls[i])
        errs.append(err)
    return errs

def errs2(amp2, off2, azs, deltaAzs):
    errs = []
    for i in range(len(azs)):
        model = off2 + amp2 * np.cos((azs[i]) * np.pi / 180.0)
        err = abs(model - deltaAzs[i])
        errs.append(err)
    return errs


In [ ]:
[amp1, off1] = result1.x
[amp2, off2] = result2.x
elErrs = errs1(amp1, off1, azs, deltaEls)
azErrs = errs2(amp2, off2, azs, deltaAzs)

In [ ]:
plt.subplot(1,2,1)
plt.hist(elErrs, bins = 50)
#plt.xlim(0, 5000)
plt.subplot(1,2,2)
plt.hist(azErrs, bins = 50)
#plt.xlim(0, 5000)

In [ ]:
def trim(azs, els, deltaAzs, deltaEls, trueAzs, trueEls, rots, times, dayObss, off1, amp1, off2, amp2, limit=100):
    elErrs = errs1(amp1, off1, azs, deltaEls)
    azErrs = errs2(amp2, off2, azs, deltaAzs)
    newAzs = []; newEls = []; newDeltaAzs = []; newDeltaEls = []; newRots = []
    newTrueAzs = []; newTrueEls = []; newTimes = []; newDayObss = []
    for i in range(len(azs)):
        #if (azErrs[i] < limit) and (elErrs[i] < limit):
        if (elErrs[i] < limit):
            newAzs.append(azs[i]); newEls.append(els[i]); newRots.append(rots[i])
            newDeltaAzs.append(deltaAzs[i]); newDeltaEls.append(deltaEls[i])
            newTrueAzs.append(trueAzs[i]); newTrueEls.append(trueEls[i]); newTimes.append(times[i])
            newDayObss.append(dayObss[i]
    print(len(newEls), len(newDeltaEls), len(newAzs), len(newDeltaAzs), len(newTrueAzs), len(newTrueEls))
    return newAzs, newEls, newDeltaAzs, newDeltaEls, newTrueAzs, newTrueEls, newRots, newTimes, newDayObss
        
    

In [ ]:
azs, els, deltaAzs, deltaEls, trueAzs, trueEls, rots, times, dayObss = trim(azs, els, deltaAzs, deltaEls, trueAzs, trueEls, rots, times, dayObss, off1, amp1, off2, amp2, limit=40)

In [ ]:
print(len(trueAzs), len(trueEls), len(els), len(deltaEls), len(azs), len(deltaAzs), len(times), len(dayObss))

In [ ]:

[amp1, off1] = result1.x
[amp2, off2] = result2.x
xs = np.linspace(-180.0, 180.0, 200)
ys1 = off1 + amp1 * np.sin((xs) * np.pi / 180.0)
ys2 = off2 + amp2 * np.cos((xs) * np.pi / 180.0)

deltaAzsCosEl = np.array(deltaAzs) * np.cos(np.array(els) * np.pi / 180.0)
plt.figure(figsize=(8,8))
plt.suptitle("LSSTCam Delta AltAz 20250527-20250608")
plt.subplots_adjust(hspace=0.3, wspace=0.7)
plt.subplot(2,2,1)
p1 = plt.scatter(els, deltaEls, c=azs, cmap=plt.cm.coolwarm)
cb1 = plt.colorbar(p1)
cb1.set_label('Az')
plt.xlabel('El')
plt.xlim(30, 85)
plt.ylabel('Delta El arcsec')
plt.subplot(2,2,2)
p2 = plt.scatter(azs, deltaEls,c=els, cmap=plt.cm.coolwarm)
cb2 = plt.colorbar(p2)
cb2.set_label('El')
plt.plot(xs, ys1, ls = '--', color='black')
plt.text(-150,-80,f"deltaEl={off1:.1f}+\n{amp1:.1f}*sin(az)")
plt.xlabel('Az')
plt.xlim(-180, 180)
plt.ylabel('Delta El arcsec')
plt.subplot(2,2,3)
p3 = plt.scatter(els, deltaAzsCosEl, c=azs, cmap=plt.cm.coolwarm)
cb3 = plt.colorbar(p3)
cb3.set_label('Az')
plt.xlabel('El')
plt.xlim(30, 85)
plt.ylabel('Delta Az*cos(El) arcsec')
plt.subplot(2,2,4)
p4 = plt.scatter(azs, deltaAzsCosEl,c=els, cmap=plt.cm.coolwarm)
plt.plot(xs, ys2, ls = '--', color='black')
plt.text(-100,-50,f"deltaAz={off2:.1f}+\n{amp2:.1f}*cos(az)")
cb4 = plt.colorbar(p4)
cb4.set_label('El')
plt.xlabel('Az')
plt.xlim(-180, 180)
plt.ylabel('Delta Az*cos(El) arcsec')
plt.savefig("/home/c/cslage/u/LSSTCam/data/Delta_AltAz_27May25-08Jun25.png")


## Get the MTMount state changes

In [ ]:
startDay = 20250527
endDay = 20250608
dayObs = startDay
enableDict = {}
while dayObs <= endDay:
    start = Time(f"{dayObsIntToString(dayObs)}T12:00:00")
    end = Time(f"{dayObsIntToString(calcNextDay(dayObs))}T12:00:00")
    states = getEfdData(
        client,
        "lsst.sal.MTMount.logevent_summaryState",
        columns=['summaryState'],
        begin=start,
        end=end
    )
    enables = states[states['summaryState'] == 2]
    print(f"There were {len(enables)} enable events on {dayObs}")
    enableDict[dayObs] = enables
    dayObs = calcNextDay(dayObs)

# Generate a tpoint input file with these AltAz errors
## This includes dates and state changes

In [ ]:
outfilename = "/home/c/cslage/u/LSSTCam/data/Tpoint_Input_Dates_14Jun25.dat"
outfile = open(outfilename, 'w')
outfile.write("!" + outfilename + "\n")
outfile.write("!Simonyi Telescope file,June 14, 2025 \n")
outfile.write(": ALTAZ\n")
outfile.write(": ROTNR\n")
outfile.write("-30 14 40.2\n")
lastDayObs = 0
for i in range(len(els)):
    dayObs = dayObss[i]
    enables = enableDict[dayObs]
    thisTime = Time(times[i], format='mjd')
    if dayObs != lastDayObs:
        lastDayObs = dayObs
        initialStateChanges = 0
        for j in range(len(enables)):
            stateChangeTime = Time(enables.index[j])
            if thisTime > stateChangeTime:
                initialStateChanges += 1
    if azs[i] > 360.0:
        azs[i] -= 360.0
    if azs[i] < 0.0:
        azs[i] += 360.0
    stateChanges = -initialStateChanges
    for j in range(len(enables)):
        stateChangeTime = Time(enables.index[j])
        if thisTime > stateChangeTime:
            stateChanges += 1
    outfile.write(f"{trueAzs[i]:.9f}\t{trueEls[i]:.6f}\t{azs[i]:.9f}\t{els[i]:.9f}\t{rots[i]:.6f}\t; {dayObs}  {stateChanges}\n")
outfile.write("END\n")
outfile.close()



In [ ]:
outfilename = "/home/c/cslage/u/LSSTCam/data/Tpoint_Input_Range_14Jun25.dat"
outfile = open(outfilename, 'w')
outfile.write("!" + outfilename + "\n")
outfile.write("!Simonyi Telescope file,June 14, 2025 \n")
outfile.write(": ALTAZ\n")
outfile.write(": ROTNR\n")
outfile.write("-30 14 40.2\n")
lastDayObs = 0
for i in range(len(els)):
    dayObs = dayObss[i]
    enables = enableDict[dayObs]
    thisTime = Time(times[i], format='mjd')
    if dayObs != lastDayObs:
        lastDayObs = dayObs
        initialStateChanges = 0
        for j in range(len(enables)):
            stateChangeTime = Time(enables.index[j])
            if thisTime > stateChangeTime:
                initialStateChanges += 1
    if azs[i] > 180.0:
        azs[i] -= 360.0
    if azs[i] < -180.0:
        azs[i] += 360.0
    if trueAzs[i] > 180.0:
        trueAzs[i] -= 360.0
    if trueAzs[i] < -180.0:
        trueAzs[i] += 360.0
    stateChanges = -initialStateChanges
    for j in range(len(enables)):
        stateChangeTime = Time(enables.index[j])
        if thisTime > stateChangeTime:
            stateChanges += 1
    outfile.write(f"{trueAzs[i]:.9f}\t{trueEls[i]:.6f}\t{azs[i]:.9f}\t{els[i]:.9f}\t{rots[i]:.6f}\t; {dayObs}  {stateChanges}\n")
outfile.write("END\n")
outfile.close()



# Generate a separate tpoint file for each dayObs and stateChange


In [ ]:
startDay = 20250527
endDay = 20250608
totalLines = 0
for thisStateChange in [0, 1, 2]:
    dayObs = startDay
    while dayObs <= endDay:
        lines = 0
        outfilename = f"/home/c/cslage/u/LSSTCam/data/Tpoint_Input_{dayObs}_{thisStateChange}.dat"
        outfile = open(outfilename, 'w')
        outfile.write("!" + outfilename + "\n")
        outfile.write("!Simonyi Telescope file,June 14, 2025 \n")
        outfile.write(": ALTAZ\n")
        outfile.write(": ROTNR\n")
        outfile.write("-30 14 40.2\n")
        lastDayObs = 0
        for i in range(len(els)):
            if dayObs != dayObss[i]:
                continue
            enables = enableDict[dayObs]
            thisTime = Time(times[i], format='mjd')
            if dayObs != lastDayObs:
                lastDayObs = dayObs
                initialStateChanges = 0
                for j in range(len(enables)):
                    stateChangeTime = Time(enables.index[j])
                    if thisTime > stateChangeTime:
                        initialStateChanges += 1
            if azs[i] > 360.0:
                azs[i] -= 360.0
            if azs[i] < 0.0:
                azs[i] += 360.0
            stateChanges = -initialStateChanges
            for j in range(len(enables)):
                stateChangeTime = Time(enables.index[j])
                if thisTime > stateChangeTime:
                    stateChanges += 1
            if stateChanges == thisStateChange:
                outfile.write(f"{trueAzs[i]:.9f}\t{trueEls[i]:.6f}\t{azs[i]:.9f}\t{els[i]:.9f}\t{rots[i]:.6f}\t; {dayObs}  {stateChanges}\n")
                lines += 1
        outfile.write("END\n")
        outfile.close()
        print(f"{dayObs}, {thisStateChange} has {lines} lines")
        totalLines += lines
        if lines < 20:
            os.remove(outfilename)
        dayObs = calcNextDay(dayObs)
print(f"TotaLines = {totalLines}")


In [ ]:
[[20250527, 0], [20250528, 0], [20250529, 0], 20250531, 0], [20250601, 0], [20250603, 0],
    [20250604, 0], [20250527, 1], [20250528, 1], [20250529, 1], [20250530, 1],
     [20250601, 1], [20250608, 1], [20250527, 2], [20250530, 2], [20250608, 2]]

## Test if that's how the AzEl coordinates respond to a rotation about one of the axes

In [ ]:
def to_xyz(az, el):
    x = np.cos(el)*np.cos(az)
    y = np.cos(el)*np.sin(az)
    z = np.sin(el)
    return (x,y,z)

def to_azel(x, y, z):
    el = np.arcsin(z)
    az = np.arctan2(y, x)    
    return az, el
    
def Ry(theta):
  return np.matrix([[ np.cos(theta), 0, np.sin(theta)],
                   [ 0           , 1, 0           ],
                   [-np.sin(theta), 0, np.cos(theta)]])

def Rx(theta):
  return np.matrix([[ 1, 0           , 0           ],
                   [ 0, np.cos(theta),-np.sin(theta)],
                   [ 0, np.sin(theta), np.cos(theta)]])


In [ ]:
# Check if we recover the initial AzEl
az = 153.0; el = 65.3
(x,y,z) = to_xyz(az*np.pi/180.0, el*np.pi/180.0)
print((x,y,z))
az2, el2 = to_azel(x,y,z)
print(az2*180.0/np.pi, el2*180.0/np.pi)

In [ ]:
test_azs = []
test_els = []
rot_azs = []
rot_els = []
test_deltaAzs = []
test_deltaEls = []
rot = -221.4 / 3600.0 * np.pi / 180.0
rx = Rx(rot)
for az in range(-175, 185, 15):
    for el in range(40, 85, 10):
        test_azs.append(az)
        test_els.append(el)
        (x,y,z) = to_xyz(az*np.pi/180.0, el*np.pi/180.0)
        out = rx.dot((x,y,z))
        out = np.array(out)[0]
        az2, el2 = to_azel(out[0], out[1], out[2])
        az2 = az2*180.0/np.pi
        el2 = el2*180.0/np.pi + 97 / 3600.0
        rot_azs.append(az2)
        rot_els.append(el2)
        test_deltaAz = (az2 - az) * 3600.0
        test_deltaEl = (el2 - el) * 3600.0
        test_deltaAzs.append(test_deltaAz * np.cos(el*np.pi/180.0))
        test_deltaEls.append(test_deltaEl)





In [ ]:
plt.figure(figsize=(8,8))
plt.suptitle("Test DeltaAz")
plt.subplots_adjust(hspace=0.3, wspace=0.7)
plt.subplot(2,2,1)
p1 = plt.scatter(test_els, test_deltaEls, c=test_azs, cmap=plt.cm.coolwarm)
cb1 = plt.colorbar(p1)
cb1.set_label('Az')
plt.xlabel('El')
plt.xlim(0,90)
plt.ylabel('Delta El arcsec')
plt.subplot(2,2,2)
p2 = plt.scatter(test_azs, test_deltaEls,c=test_els, cmap=plt.cm.coolwarm)
cb2 = plt.colorbar(p2)
cb2.set_label('El')
plt.xlabel('Az')
plt.xlim(-180, 180)
plt.ylabel('Delta El arcsec')
plt.subplot(2,2,3)
p3 = plt.scatter(test_els, test_deltaAzs, c=test_azs, cmap=plt.cm.coolwarm)
cb3 = plt.colorbar(p3)
cb3.set_label('Az')
plt.xlabel('El')
plt.xlim(0,90)
plt.ylabel('Delta Az*cos(El) arcsec')
plt.subplot(2,2,4)
p4 = plt.scatter(test_azs, test_deltaAzs,c=test_els, cmap=plt.cm.coolwarm)
cb4 = plt.colorbar(p4)
cb4.set_label('El')
plt.xlabel('Az')
plt.xlim(-180, 180)
plt.ylabel('Delta Az*cos(El) arcsec')
plt.savefig("/home/c/cslage/u/LSSTCam/data/Test_Delta_AltAz_13Jun25.png")


# Generate a tpoint input file with the model AltAz errors

In [ ]:
outfilename = "/home/c/cslage/u/LSSTCam/data/Tpoint_Model_13Jun25.dat"
outfile = open(outfilename, 'w')
outfile.write("!" + outfilename + "\n")
outfile.write("!Simonyi Telescope file,June 9, 2025 \n")
outfile.write(": ALTAZ\n")
outfile.write(": ROTNR\n")
outfile.write("-30 14 40.2\n")
rot = 0.0
for i in range(len(els)):
    if azs[i] > 360.0:
        azs[i] -= 360.0
    if azs[i] < 0.0:
        azs[i] += 360.0
    outfile.write(f"{test_azs[i]:.9f}\t{test_els[i]:.6f}\t{rot_azs[i]:.9f}\t{rot_els[i]:.9f}\t{rot:.6f}\n")
outfile.write("END\n")
outfile.close()

# Running Tpoint n this file recovered these exact values: AW = 221.4, IE = -97

In [ ]:
rad_azs = np.array(azs) * np.pi / 180.0

fig=plt.figure(figsize=(8,8))
ax1 = plt.subplot(111, projection='polar')
ax1.set_title(f"Sky coverage - LSSTCam 20250527-20250608 Trimmed")
ax1.invert_yaxis()
ax1.set_rlim(90,0)
ax1.scatter (rad_azs, els)
plt.savefig("/home/c/cslage/u/LSSTCam/data/Sky_Coverage_Pointing_27May25-08Jun25.png")

In [ ]:
butler = Butler('/sdf/group/rubin/repo/main', collections=["LSSTComCam/raw/all","LSSTComCam/calib"])
instrument = 'LSSTComCam'

def DeltaAltAz (el, az):
    # This calculates the offset due to the pointing model
    # as extracted from ComCam images.
    deltaEl = 125.4 - 187.9 * np.sin(az * np.pi / 180.0)
    deltaAz = 222.0 + 348.1 * np.cos(az * np.pi / 360.0)
    elPrime = el + deltaEl / 3600.0
    azPrime = az + deltaAz / 3600.0
    return [elPrime, azPrime]

def CalculateDrift(expId):
    rawExp = butler.get('raw', detector=4, exposure=expId, instrument=instrument)
    md = rawExp.getMetadata()
    filter = md['FILTBAND']
    pressure = md['PRESSURE'] * u.pascal
    temperature = md['AIRTEMP'] * u.Celsius
    hum = md['HUMIDITY']
    time1 = Time(md['MJD-BEG'], format='mjd', scale='tai')
    time2 = Time(md['MJD-END'], format='mjd', scale='tai')
    ra = md['RASTART']
    dec = md['DECSTART']
    skyLocation = SkyCoord(ra*u.deg, dec*u.deg)
    wl = wavelengths[filter] * u.angstrom
    altAz1 = AltAz(obstime=time1, location=SIMONYI_LOCATION, pressure=pressure, 
                 temperature=temperature, relative_humidity=hum, obswl=wl)
    altAz2 = AltAz(obstime=time2, location=SIMONYI_LOCATION, pressure=pressure, 
                 temperature=temperature, relative_humidity=hum, obswl=wl)
    obsAltAz1 = skyLocation.transform_to(altAz1)
    obsAltAz2 = skyLocation.transform_to(altAz2)
    # 1 is at the beginning of the exposure, 2 is at the end
    # el, az are the actual values, prime values reflect the pointing model
    # These are all in degrees
    el1 = obsAltAz1.alt.deg
    az1 = obsAltAz1.az.deg
    el2 = obsAltAz2.alt.deg
    az2 = obsAltAz2.az.deg
    [elPrime1, azPrime1] = DeltaAltAz (el1, az1)
    [elPrime2, azPrime2] = DeltaAltAz (el2, az2)
    # Change values are the change from the beginning to the end of the exposure, in arcseconds
    azChange = (az2 - az1) * 3600.0
    elChange = (el2 - el1) * 3600.0
    azPrimeChange = (azPrime2 - azPrime1) * 3600.0
    elPrimeChange = (elPrime2 - elPrime1) * 3600.0
    azDrift = azChange - azPrimeChange
    elDrift = elChange - elPrimeChange
    print(f" For {expId}, Azimuth drift = {azDrift:.2f} arcseconds, Elevation drift = {elDrift:.2f} arcseconds")
    return

CalculateDrift(2024120700551)

In [ ]:
plt.scatter(azs, els)

In [ ]:
azs[0]